# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [15]:
# import numpy and pandas
import numpy as np
import pandas as pd
from scipy import stats as st

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
# Your code here:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [9]:
# Your code here:
len(pokemon[pokemon['Legendary'] == True])
len(pokemon[pokemon['Legendary'] == False])

735

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [14]:
# Your code here:
print(pokemon['Total'][pokemon['Legendary'] == True].mean(),
pokemon['Total'][pokemon['Legendary'] == True].std(),
pokemon['Total'][pokemon['Legendary'] == False].mean(),
pokemon['Total'][pokemon['Legendary'] == False].std())

637.3846153846154 60.93738905315346 417.21360544217686 106.76041745713022


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [18]:
# Your code here:
st.ttest_ind(pokemon['Total'][pokemon['Legendary'] == True],pokemon['Total'][pokemon['Legendary'] == False], equal_var=False)

Ttest_indResult(statistic=25.8335743895517, pvalue=9.357954335957446e-47)

What do you conclude from this test? Write your conclusions below.

In [56]:
# Your conclusions here:

#It is therefore very unlikely that the two samples are identical, or everelated 

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [43]:
# Your code here:
type_1_count_all = [item for item in pokemon['Type 1']]


type_1_count_aller = [[x,type_1_count_all.count(x)] for x in type_1_count_all]

type_1_count = []
for item in type_1_count_aller:
    if item not in type_1_count:
        type_1_count.append(item)
type_1_count

[['Grass', 70],
 ['Fire', 52],
 ['Water', 112],
 ['Bug', 69],
 ['Normal', 98],
 ['Poison', 28],
 ['Electric', 44],
 ['Ground', 32],
 ['Fairy', 17],
 ['Fighting', 27],
 ['Psychic', 57],
 ['Rock', 44],
 ['Ghost', 32],
 ['Ice', 24],
 ['Dragon', 32],
 ['Dark', 31],
 ['Steel', 27],
 ['Flying', 4]]

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [51]:
# Your code here:
for item in pokemon['Type 1'].unique():
    print(item, pokemon['Total'][pokemon['Type 1'] == item].mean(), pokemon['Total'][pokemon['Type 1'] == item].std())


Grass 421.14285714285717 106.6506263819414
Fire 458.0769230769231 109.76049615338435
Water 430.45535714285717 113.18826606431458
Bug 378.92753623188406 117.8752230165166
Normal 401.68367346938777 115.73303771765518
Poison 399.14285714285717 92.35868656562297
Electric 443.40909090909093 105.72195173294949
Ground 437.5 123.91308087890974
Fairy 413.1764705882353 123.78168043682679
Fighting 416.44444444444446 102.46437780003703
Psychic 475.94736842105266 139.0266445496576
Rock 453.75 108.05922686947147
Ghost 439.5625 110.07268493019481
Ice 433.4583333333333 108.28102694886786
Dragon 550.53125 146.26753754788714
Dark 445.741935483871 109.12621675287613
Steel 487.7037037037037 115.42090692325094
Flying 485.0 161.40012391568973


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [48]:
# Your code here:
for item in pokemon['Type 1'].unique():
    print(item, st.ttest_1samp(pokemon['Total'][pokemon['Type 1'] == item], pokemon['Total'][pokemon['Type 1'] == 'Water'].mean()).pvalue)

Grass 0.467525432334701
Fire 0.0754539470916475
Water 1.0
Bug 0.0005422774154009478
Normal 0.015620015674898775
Poison 0.08402265143226227
Electric 0.42083952953464066
Ground 0.749913812337193
Fairy 0.5729251293048644
Fighting 0.4837078337299058
Psychic 0.01656566934710942
Rock 0.1599578941702502
Ghost 0.6430341649812233
Ice 0.8931107536296519
Dragon 5.944853700205225e-05
Dark 0.4415356166730098
Steel 0.015981925860619164
Flying 0.5475474923657161


Write your conclusion below.

In [57]:
# Your conclusions here:
#The mean is closely related to almost all other pokemon except fire, psychic, steel, dragon and bug

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [58]:
# Your code here:
from scipy.stats import ttest_rel
ttest_rel(pokemon['Attack'], pokemon['Defense'])

Ttest_relResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

Describe the results of the test in the cell below.

In [60]:
# Your conclusions here:
#We can reject the hypothesis that the two are related

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [63]:
# Your code here:
ttest_rel(pokemon['Sp. Atk'],pokemon['Sp. Def'])

Ttest_relResult(statistic=0.853986188453353, pvalue=0.3933685997548122)

Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:

#We cannot reject the hypothesis that there is no difference

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [65]:
# Your code here:
from scipy.stats import ttest_1samp
ttest_1samp((pokemon['Sp. Atk'] - pokemon['Sp. Def']), 0)

Ttest_1sampResult(statistic=0.853986188453353, pvalue=0.3933685997548122)

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [ ]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [ ]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [ ]:
# Your answer here:

